# OWON SPM SCPI Native Serial Communication

This notebook demonstrates how to communicate with an OWON SPM instrument using SCPI commands over `/dev/ttyUSB0` natively (without VISA), using Python's `pyserial` library.

In [1]:
#!pip install pyserial

import serial

# Global serial handle
SER_HANDLE = None

def get_serial():
    global SER_HANDLE
    if SER_HANDLE is None or not SER_HANDLE.is_open:
        SER_HANDLE = serial.Serial(
            port='/dev/ttyUSB0',
            baudrate=115200,
            bytesize=serial.EIGHTBITS,
            parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE,
            timeout=0.5
        )
        print('Serial port opened:', SER_HANDLE.port)
    return SER_HANDLE

# Open serial port at notebook start
get_serial()

Serial port opened: /dev/ttyUSB0


Serial<id=0x7fa214246890, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)

### 2. Send SCPI Commands to OWON SPM

In [2]:
def send_scpi_command(command, silent = True):
    ser = get_serial()
    ser.write((command + '\n').encode('ascii'))
    if not silent:
        print(f'Sent: {command}')

def read_scpi_response(silent=True):
    ser = get_serial()
    response = ser.readline().decode('ascii', errors='replace').strip()
    if not silent:
        print('Received:', response)
    return response

send_scpi_command('*IDN?')
read_scpi_response()

def measure_it(command='MEAS:ALL:INFO?'):
    send_scpi_command(command)
    response = read_scpi_response()
    return response

# Example: Measure DC current
current = measure_it("MEAS:CURRENT:DC?")
print(f'Current Measurement: {current}')

Current Measurement: 0.000


In [3]:
def extract_current_measurements(text):
    parts = text.split(',')
    floats = [float(x) for x in parts[:3]]
    return floats

# Extract first 3 floats from a comma-separated string
text = '3.214,1.004,3.230,OFF,OFF,OFF,2'
text = measure_it()

In [4]:
import time
from datetime import datetime
measurements = []
while True:
    current_measurements = extract_current_measurements(measure_it())
    timestamp = datetime.now().isoformat()
    row = [timestamp] + current_measurements
    measurements.append(row)
    print(f'Measurements: {row}')

    time.sleep(1)

Measurements: ['2025-08-11T14:18:30.791630', 0.0, 0.0, 0.0]
Measurements: ['2025-08-11T14:18:31.804141', 0.0, 0.0, 0.0]
Measurements: ['2025-08-11T14:18:31.804141', 0.0, 0.0, 0.0]
Measurements: ['2025-08-11T14:18:32.816697', 0.0, 0.0, 0.0]
Measurements: ['2025-08-11T14:18:32.816697', 0.0, 0.0, 0.0]
Measurements: ['2025-08-11T14:18:33.829262', 0.0, 0.0, 0.0]
Measurements: ['2025-08-11T14:18:33.829262', 0.0, 0.0, 0.0]
Measurements: ['2025-08-11T14:18:34.842046', 0.0, 0.0, 0.0]
Measurements: ['2025-08-11T14:18:34.842046', 0.0, 0.0, 0.0]


KeyboardInterrupt: 

### activate charging


In [ ]:
def set_power_supply(voltage, current):
    send_scpi_command(f'VOLT {voltage}')
    read_scpi_response()
    send_scpi_command(f'CURR {current}')
    read_scpi_response()
    send_scpi_command('OUTP ON')
    read_scpi_response()
    print(f'Set voltage to {voltage} V and current to {current} A, output enabled.')

# Example: Set to 3.6V and 1A, output ON
set_power_supply(3., 3.0)

In [ ]:
send_scpi_command('OUTP?')  # Turn off output after use
send_scpi_command('CURR:LIM?') 
print(read_scpi_response())
print(read_scpi_response())
send_scpi_command('OUTP OFF')

In [ ]:
if SER_HANDLE is not None and SER_HANDLE.is_open:
    SER_HANDLE.close()  # Close the serial port when done

In [ ]:
import pandas as pd

# Convert measurements to a DataFrame for easier handling
df = pd.DataFrame(measurements, columns=["timestamp"] + [f"measurement_{i}" for i in range(len(measurements[0]) - 1)])
#plot df
import matplotlib.pyplot as plt

df.set_index("timestamp", inplace=True)
df.plot()
plt.show()

In [6]:
print(measure_it("CONFigure:ALL?"))
print(measure_it("MEAS:ALL:INFO?"))
#print(measure_current("CONF?"))

VOLT:DC,+02.952V,AUTO,20V
0.000,0.000,0.000,OFF,OFF,OFF,0
